In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import numpy as np
import pandas as pd

from utils import get_differences, read_csv_dataset, create_mcar, verify_error_pct

In [16]:
from tab_err import ErrorModel, error_type
from tab_err.error_mechanism import EAR, ENAR, ECAR
from tab_err.error_type import ErrorTypeConfig
from tab_err.api import mid_level, MidLevelConfig

## beers

In [17]:
df_clean = read_csv_dataset('../data/beers/clean.csv')
df_dirty = read_csv_dataset('../data/beers/dirty.csv')


df_clean.to_csv(f'../export_data/beers/clean.csv', index=False)
df_dirty.to_csv(f'../export_data/beers/beers_original.csv', index=False)

error_percentages = (df_dirty != df_clean).sum() / df_dirty.shape[0]

# In the original error distribution, all values in column 'ounces' are erroneous.
# However, HoloClean will automatically drop empty columns, preventing us from measuring
# a benchmark. We thus move two errors from the 'ounces' colum into the 'city' colum.
error_percentages['ounces'] = 1 - 2/df_dirty.shape[0]
error_percentages['city'] = error_percentages['city'] + 2/df_dirty.shape[0]

create_mcar('beers', df_clean, error_percentages)

df_clean['ounces'] = df_clean['ounces'].astype('category')
df_clean['city'] = df_clean['city'].astype('category')
df_clean['state'] = df_clean['state'].astype('category')

Saved MCAR dataset beers iteration 0
Saved MCAR dataset beers iteration 1
Saved MCAR dataset beers iteration 2
Saved MCAR dataset beers iteration 3
Saved MCAR dataset beers iteration 4
Saved MCAR dataset beers iteration 5
Saved MCAR dataset beers iteration 6
Saved MCAR dataset beers iteration 7
Saved MCAR dataset beers iteration 8
Saved MCAR dataset beers iteration 9


In [18]:
scenario_name = 'beers_scenario'

for i in range(10):
    base_seed = i * 1000  # Using 1000 ensures no overlap between iterations
    seeds = [base_seed + 1, base_seed + 2, base_seed + 3, base_seed + 4, base_seed + 5]
    
    mid_lvl_config = {
        'ounces': [ErrorModel(ECAR(seed=seeds[0]), error_type.CategorySwap(seed=seeds[0]), error_percentages['ounces'])],  # ECAR because error_percentage is 1.0 in 'ounces', mechanism doesn't matter
        'abv': [ErrorModel(EAR(condition_to_column='brewery-name', seed=seeds[1]), error_type.Extraneous(seed=seeds[1]), error_percentages['abv'])],
        'ibu': [ErrorModel(EAR(condition_to_column='brewery-name', seed=seeds[2]), error_type.Extraneous(seed=seeds[2]), error_percentages['ibu'])],
        'city': [ErrorModel(EAR(condition_to_column='brewery-name', seed=seeds[3]), error_type.CategorySwap(seed=seeds[3]), error_percentages['city'])],
        'state': [ErrorModel(EAR(condition_to_column='brewery-name', seed=seeds[4]), error_type.CategorySwap(seed=seeds[4]), error_percentages['state'])],
    }
    scenario = MidLevelConfig(mid_lvl_config)
    df_corrupted, error_mask = mid_level.create_errors(df_clean, scenario)
    df_corrupted.to_csv(f'../export_data/beers/{scenario_name}_{i}.csv', index=False)

/Users/philipp/Library/Caches/pypoetry/virtualenvs/error-paper-mf3FH1nM-py3.10/lib/python3.10/site-packages/tab_err/error_type/_error_type.py:67: UserWarning: self.config.extraneous_value_template is not set. Choosing a random string augmentation.
  return self._apply(data, error_mask, column)
/Users/philipp/Library/Caches/pypoetry/virtualenvs/error-paper-mf3FH1nM-py3.10/lib/python3.10/site-packages/tab_err/error_type/_error_type.py:67: UserWarning: self.config.extraneous_value_template is not set. Choosing a random string augmentation.
  return self._apply(data, error_mask, column)
/Users/philipp/Library/Caches/pypoetry/virtualenvs/error-paper-mf3FH1nM-py3.10/lib/python3.10/site-packages/tab_err/error_type/_error_type.py:67: UserWarning: self.config.extraneous_value_template is not set. Choosing a random string augmentation.
  return self._apply(data, error_mask, column)
/Users/philipp/Library/Caches/pypoetry/virtualenvs/error-paper-mf3FH1nM-py3.10/lib/python3.10/site-packages/tab_err

In [19]:
for i in range(10):
    verify_error_pct('../export_data/beers/clean.csv', f'../export_data/beers/beers_missing_ecar_{i}.csv', error_percentages)
for i in range(10):
    verify_error_pct('../export_data/beers/clean.csv', f'../export_data/beers/beers_scenario_{i}.csv', error_percentages)

beers_missing_ecar_0.csv deviates from intended error_pct by absolute 0.0%.
beers_missing_ecar_1.csv deviates from intended error_pct by absolute 0.0%.
beers_missing_ecar_2.csv deviates from intended error_pct by absolute 0.0%.
beers_missing_ecar_3.csv deviates from intended error_pct by absolute 0.0%.
beers_missing_ecar_4.csv deviates from intended error_pct by absolute 0.0%.
beers_missing_ecar_5.csv deviates from intended error_pct by absolute 0.0%.
beers_missing_ecar_6.csv deviates from intended error_pct by absolute 0.0%.
beers_missing_ecar_7.csv deviates from intended error_pct by absolute 0.0%.
beers_missing_ecar_8.csv deviates from intended error_pct by absolute 0.0%.
beers_missing_ecar_9.csv deviates from intended error_pct by absolute 0.0%.
beers_scenario_0.csv deviates from intended error_pct by absolute 0.0%.
beers_scenario_1.csv deviates from intended error_pct by absolute 0.0%.
beers_scenario_2.csv deviates from intended error_pct by absolute 0.0%.
beers_scenario_3.csv dev

## bridges

In [20]:
df_clean = read_csv_dataset('../data/bridges/clean.csv')
df_dirty = read_csv_dataset('../data/bridges/bridges_3_1.csv')

df_clean.to_csv(f'../export_data/bridges/clean.csv', index=False)
df_dirty.to_csv(f'../export_data/bridges/bridges_original.csv', index=False)

error_percentages = (df_dirty != df_clean).sum() / df_dirty.shape[0]

create_mcar('bridges', df_clean, error_percentages)

df_clean["PURPOSE"] = df_clean["PURPOSE"].astype('string')
df_clean["LOCATION"] = df_clean["LOCATION"].astype('category')
df_clean["LANES"] = df_clean["LANES"].astype('category')
df_clean["CLEAR-G"] = df_clean["CLEAR-G"].astype('string')
df_clean["T-OR-D"] = df_clean["T-OR-D"].astype('category')
df_clean["MATERIAL"] = df_clean["MATERIAL"].astype('category')
df_clean["SPAN"] = df_clean["SPAN"].astype('category')
df_clean["REL-L"] = df_clean["REL-L"].astype('category')
df_clean["TYPE"] = df_clean["TYPE"].astype('category')



Saved MCAR dataset bridges iteration 0
Saved MCAR dataset bridges iteration 1
Saved MCAR dataset bridges iteration 2
Saved MCAR dataset bridges iteration 3
Saved MCAR dataset bridges iteration 4
Saved MCAR dataset bridges iteration 5
Saved MCAR dataset bridges iteration 6
Saved MCAR dataset bridges iteration 7
Saved MCAR dataset bridges iteration 8
Saved MCAR dataset bridges iteration 9


In [21]:
scenario_name = 'bridges_scenario'

for i in range(10):
    base_seed = i * 1000  # Using 1000 ensures no overlap between iterations
    seeds = [base_seed + (j+1) for j in range(len(df_clean.columns))]
    
    mid_lvl_config = {
        "LOCATION": [ErrorModel(ENAR(seed=seeds[0]), error_type.CategorySwap(seed=seeds[0]), error_percentages['LOCATION'])],
        "ERECTED": [ErrorModel(EAR(condition_to_column='LOCATION', seed=seeds[1]), error_type.Extraneous(seed=seeds[1]), error_percentages['ERECTED'])],
        "PURPOSE": [ErrorModel(EAR(condition_to_column='LOCATION', seed=seeds[2]), error_type.Extraneous({'extraneous_value_template': 'Purpose: {value}'}, seed=seeds[2]), error_percentages['PURPOSE'])],
        "LENGTH": [ErrorModel(ENAR(seed=seeds[3]), error_type.Extraneous(seed=seeds[3]), error_percentages['LENGTH'])],
        "LANES": [ErrorModel(EAR(condition_to_column='PURPOSE', seed=seeds[4]), error_type.CategorySwap(seed=seeds[4]), error_percentages['LANES'])],
        "CLEAR-G": [ErrorModel(ENAR(seed=seeds[5]), error_type.MissingValue({'missing_value': '?'}, seed=seeds[5]), error_percentages['CLEAR-G'])],
        "T-OR-D": [ErrorModel(EAR(condition_to_column='PURPOSE', seed=seeds[6]), error_type.CategorySwap(seed=seeds[6]), error_percentages['T-OR-D'])],
        "MATERIAL": [ErrorModel(EAR(condition_to_column='ERECTED', seed=seeds[7]), error_type.CategorySwap(seed=seeds[7]), error_percentages['MATERIAL'])],
        "SPAN": [ErrorModel(EAR(condition_to_column='PURPOSE', seed=seeds[8]), error_type.CategorySwap(seed=seeds[8]), error_percentages['SPAN'])],
        "REL-L": [ErrorModel(EAR(condition_to_column='PURPOSE', seed=seeds[9]), error_type.CategorySwap(seed=seeds[9]), error_percentages['REL-L'])],
        "TYPE": [ErrorModel(EAR(condition_to_column='PURPOSE', seed=seeds[10]), error_type.CategorySwap(seed=seeds[10]), error_percentages['TYPE'])],
    }

    scenario = MidLevelConfig(mid_lvl_config)
    df_corrupted, error_mask = mid_level.create_errors(df_clean, scenario)
    df_corrupted.to_csv(f'../export_data/bridges/{scenario_name}_{i}.csv', index=False)

/Users/philipp/Library/Caches/pypoetry/virtualenvs/error-paper-mf3FH1nM-py3.10/lib/python3.10/site-packages/tab_err/error_type/_error_type.py:67: UserWarning: self.config.extraneous_value_template is not set. Choosing a random string augmentation.
  return self._apply(data, error_mask, column)
/Users/philipp/Library/Caches/pypoetry/virtualenvs/error-paper-mf3FH1nM-py3.10/lib/python3.10/site-packages/tab_err/error_type/_error_type.py:67: UserWarning: self.config.extraneous_value_template is not set. Choosing a random string augmentation.
  return self._apply(data, error_mask, column)
/Users/philipp/Library/Caches/pypoetry/virtualenvs/error-paper-mf3FH1nM-py3.10/lib/python3.10/site-packages/tab_err/error_type/_error_type.py:67: UserWarning: self.config.extraneous_value_template is not set. Choosing a random string augmentation.
  return self._apply(data, error_mask, column)
/Users/philipp/Library/Caches/pypoetry/virtualenvs/error-paper-mf3FH1nM-py3.10/lib/python3.10/site-packages/tab_err

In [22]:
for i in range(10):
    verify_error_pct('../export_data/bridges/clean.csv', f'../export_data/bridges/bridges_missing_ecar_{i}.csv', error_percentages)
for i in range(10):
    verify_error_pct('../export_data/bridges/clean.csv', f'../export_data/bridges/bridges_scenario_{i}.csv', error_percentages)

bridges_missing_ecar_0.csv deviates from intended error_pct by absolute 0.9259259259259258%.
bridges_missing_ecar_1.csv deviates from intended error_pct by absolute 1.8518518518518516%.
bridges_missing_ecar_2.csv deviates from intended error_pct by absolute 2.7777777777777777%.
bridges_missing_ecar_3.csv deviates from intended error_pct by absolute 0.0%.
bridges_missing_ecar_4.csv deviates from intended error_pct by absolute 0.9259259259259258%.
bridges_missing_ecar_5.csv deviates from intended error_pct by absolute 1.8518518518518516%.
bridges_missing_ecar_6.csv deviates from intended error_pct by absolute 1.8518518518518516%.
bridges_missing_ecar_7.csv deviates from intended error_pct by absolute 4.62962962962963%.
bridges_missing_ecar_8.csv deviates from intended error_pct by absolute 1.8518518518518516%.
bridges_missing_ecar_9.csv deviates from intended error_pct by absolute 1.8518518518518516%.
bridges_scenario_0.csv deviates from intended error_pct by absolute 0.0%.
bridges_scena

## cars

In [23]:
df_clean = read_csv_dataset('../data/cars/clean.csv')
df_dirty = read_csv_dataset('../data/cars/cars_3_1.csv')

df_clean.to_csv(f'../export_data/cars/clean.csv', index=False)
df_dirty.to_csv(f'../export_data/cars/cars_original.csv', index=False)

error_percentages = (df_dirty != df_clean).sum() / df_dirty.shape[0]
create_mcar('cars', df_clean, error_percentages)

df_clean['Cylinders'] = df_clean['Cylinders'].astype('category')
df_clean['Acceleration'] = df_clean['Acceleration'].astype('category')
df_clean['Model'] = df_clean['Model'].astype('category')
df_clean['Origin'] = df_clean['Origin'].astype('category')
df_clean['Displacement'] = df_clean['Displacement'].astype('Float64')
df_clean['Horsepower'] = df_clean['Horsepower'].astype('Float64')
df_clean['Weight'] = df_clean['Weight'].astype('Float64')


Saved MCAR dataset cars iteration 0
Saved MCAR dataset cars iteration 1
Saved MCAR dataset cars iteration 2
Saved MCAR dataset cars iteration 3
Saved MCAR dataset cars iteration 4
Saved MCAR dataset cars iteration 5
Saved MCAR dataset cars iteration 6
Saved MCAR dataset cars iteration 7
Saved MCAR dataset cars iteration 8
Saved MCAR dataset cars iteration 9


In [24]:
scenario_name = 'cars_scenario'

for i in range(10):
    base_seed = i * 1000  # Using 1000 ensures no overlap between iterations
    seeds = [base_seed + (j+1) for j in range(len(df_clean.columns))]
    
    mid_lvl_config = {
        "Cylinders": [ErrorModel(ENAR(seed=seeds[0]), error_type.CategorySwap(), error_percentages['Cylinders'])],
        "Displacement": [ErrorModel(EAR(condition_to_column='Horsepower', seed=seeds[1]), error_type.Outlier(), error_percentages['Displacement'])],
        "Horsepower": [ErrorModel(ENAR(seed=seeds[2]), error_type.Outlier(), error_percentages['Horsepower'])],
        "Weight": [ErrorModel(EAR(condition_to_column="Horsepower", seed=seeds[3]), error_type.Outlier(), error_percentages['Weight'])],
        "Acceleration": [ErrorModel(EAR(condition_to_column="Horsepower", seed=seeds[4]), error_type.CategorySwap(), error_percentages['Acceleration'])],
        "Model": [ErrorModel(EAR(condition_to_column="Origin", seed=seeds[5]), error_type.CategorySwap(), error_percentages['Model'])],
        "Origin": [ErrorModel(ENAR(seed=seeds[6]), error_type.CategorySwap(), error_percentages['Origin'])],
    }

    scenario = MidLevelConfig(mid_lvl_config)
    df_corrupted, error_mask = mid_level.create_errors(df_clean, scenario)
    df_corrupted.to_csv(f'../export_data/cars/{scenario_name}_{i}.csv', index=False)

In [25]:
for i in range(10):
    verify_error_pct('../export_data/cars/clean.csv', f'../export_data/cars/cars_missing_ecar_{i}.csv', error_percentages)
for i in range(10):
    verify_error_pct('../export_data/cars/clean.csv', f'../export_data/cars/cars_scenario_{i}.csv', error_percentages)

cars_missing_ecar_0.csv deviates from intended error_pct by absolute 0.0%.
cars_missing_ecar_1.csv deviates from intended error_pct by absolute 0.0%.
cars_missing_ecar_2.csv deviates from intended error_pct by absolute 0.0%.
cars_missing_ecar_3.csv deviates from intended error_pct by absolute 0.0%.
cars_missing_ecar_4.csv deviates from intended error_pct by absolute 0.0%.
cars_missing_ecar_5.csv deviates from intended error_pct by absolute 0.0%.
cars_missing_ecar_6.csv deviates from intended error_pct by absolute 0.0%.
cars_missing_ecar_7.csv deviates from intended error_pct by absolute 0.0%.
cars_missing_ecar_8.csv deviates from intended error_pct by absolute 0.0%.
cars_missing_ecar_9.csv deviates from intended error_pct by absolute 0.0%.
cars_scenario_0.csv deviates from intended error_pct by absolute 0.0%.
cars_scenario_1.csv deviates from intended error_pct by absolute 0.0%.
cars_scenario_2.csv deviates from intended error_pct by absolute 0.0%.
cars_scenario_3.csv deviates from int

## flights

In [26]:
df_clean = read_csv_dataset('../data/flights/clean.csv')
df_dirty = read_csv_dataset('../data/flights/dirty.csv')

df_clean.to_csv(f'../export_data/flights/clean.csv', index=False)
df_dirty.to_csv(f'../export_data/flights/flights_original.csv', index=False)

error_percentages = (df_dirty != df_clean).sum() / df_dirty.shape[0]
create_mcar('flights', df_clean, error_percentages)

df_clean['sched_dep_time'] = df_clean['sched_dep_time'].astype('category')
df_clean['act_dep_time'] = df_clean['act_dep_time'].astype('category')
df_clean['sched_arr_time'] = df_clean['sched_arr_time'].astype('category')
df_clean['act_arr_time'] = df_clean['act_arr_time'].astype('category')


Saved MCAR dataset flights iteration 0
Saved MCAR dataset flights iteration 1
Saved MCAR dataset flights iteration 2
Saved MCAR dataset flights iteration 3
Saved MCAR dataset flights iteration 4
Saved MCAR dataset flights iteration 5
Saved MCAR dataset flights iteration 6
Saved MCAR dataset flights iteration 7
Saved MCAR dataset flights iteration 8
Saved MCAR dataset flights iteration 9


In [27]:
scenario_name = 'flights_scenario'

for i in range(10):
    base_seed = i * 1000  # Using 1000 ensures no overlap between iterations
    seeds = [base_seed + (j+1) for j in range(len(df_clean.columns))]
    
    mid_lvl_config = {
        "sched_dep_time": [ErrorModel(EAR(condition_to_column='flight', seed=seeds[0]), error_type.CategorySwap(), error_percentages['sched_dep_time'])],
        "act_dep_time": [ErrorModel(EAR(condition_to_column='flight', seed=seeds[1]), error_type.CategorySwap(), error_percentages['act_dep_time'])],
        "sched_arr_time": [ErrorModel(EAR(condition_to_column='flight', seed=seeds[2]), error_type.CategorySwap(), error_percentages['sched_arr_time'])],
        "act_arr_time": [ErrorModel(EAR(condition_to_column='flight', seed=seeds[3]), error_type.CategorySwap(), error_percentages['act_arr_time'])],
    }

    scenario = MidLevelConfig(mid_lvl_config)
    df_corrupted, error_mask = mid_level.create_errors(df_clean, scenario)
    df_corrupted.to_csv(f'../export_data/flights/{scenario_name}_{i}.csv', index=False)

In [28]:
for i in range(10):
    verify_error_pct('../export_data/flights/clean.csv', f'../export_data/flights/flights_missing_ecar_{i}.csv', error_percentages)
for i in range(10):
    verify_error_pct('../export_data/flights/clean.csv', f'../export_data/flights/flights_scenario_{i}.csv', error_percentages)

flights_missing_ecar_0.csv deviates from intended error_pct by absolute 0.04208754208753218%.
flights_missing_ecar_1.csv deviates from intended error_pct by absolute 0.04208754208753218%.
flights_missing_ecar_2.csv deviates from intended error_pct by absolute 0.04208754208753218%.
flights_missing_ecar_3.csv deviates from intended error_pct by absolute 0.04208754208753218%.
flights_missing_ecar_4.csv deviates from intended error_pct by absolute 0.04208754208753218%.
flights_missing_ecar_5.csv deviates from intended error_pct by absolute 0.04208754208753218%.
flights_missing_ecar_6.csv deviates from intended error_pct by absolute 0.04208754208753218%.
flights_missing_ecar_7.csv deviates from intended error_pct by absolute 0.04208754208753218%.
flights_missing_ecar_8.csv deviates from intended error_pct by absolute 0.04208754208753218%.
flights_missing_ecar_9.csv deviates from intended error_pct by absolute 0.04208754208753218%.
flights_scenario_0.csv deviates from intended error_pct by a

## food

In [29]:
df_clean = read_csv_dataset('../data/food/clean.csv').iloc[:1000, :]
df_dirty = read_csv_dataset('../data/food/dirty.csv').iloc[:1000, :]

df_clean.to_csv(f'../export_data/food/clean.csv', index=False)
df_dirty.to_csv(f'../export_data/food/food_original.csv', index=False)

error_percentages = (df_dirty != df_clean).sum() / df_dirty.shape[0]
create_mcar('food', df_clean, error_percentages)

df_clean['city'] = df_clean['city'].astype('category')
df_clean['state'] = df_clean['state'].astype('string')
df_clean['facilitytype'] = df_clean['facilitytype'].astype('category')
df_clean['address'] = df_clean['address'].astype('category')
df_clean['zip'] = df_clean['zip'].astype('category')


Saved MCAR dataset food iteration 0
Saved MCAR dataset food iteration 1
Saved MCAR dataset food iteration 2
Saved MCAR dataset food iteration 3
Saved MCAR dataset food iteration 4
Saved MCAR dataset food iteration 5
Saved MCAR dataset food iteration 6
Saved MCAR dataset food iteration 7
Saved MCAR dataset food iteration 8
Saved MCAR dataset food iteration 9


In [30]:
scenario_name = 'food_scenario'

for i in range(10):
    base_seed = i * 1000  # Using 1000 ensures no overlap between iterations
    seeds = [base_seed + (j+1) for j in range(len(df_clean.columns))]
    
    mid_lvl_config = {
        "city": [ErrorModel(EAR(condition_to_column='inspectiontype', seed=seeds[0]), error_type.CategorySwap(), error_percentages['city'])],
        "state": [ErrorModel(EAR(condition_to_column='inspectiontype', seed=seeds[1]), error_type.Replace({'replace_what': 'IL', 'replace_with': 'WS'}), error_percentages['state'])],
        "facilitytype": [ErrorModel(EAR(condition_to_column='inspectiontype', seed=seeds[2]), error_type.CategorySwap(), error_percentages['facilitytype'])],
        "address": [ErrorModel(EAR(condition_to_column='inspectiontype', seed=seeds[3]), error_type.CategorySwap(), error_percentages['address'])],
        "zip": [ErrorModel(EAR(condition_to_column='inspectiontype', seed=seeds[4]), error_type.CategorySwap(), error_percentages['zip'])],
    }

    scenario = MidLevelConfig(mid_lvl_config)
    df_corrupted, error_mask = mid_level.create_errors(df_clean, scenario)
    df_corrupted.to_csv(f'../export_data/food/{scenario_name}_{i}.csv', index=False)

In [31]:
for i in range(10):
    verify_error_pct('../export_data/food/clean.csv', f'../export_data/food/food_missing_ecar_{i}.csv', error_percentages)
for i in range(10):
    verify_error_pct('../export_data/food/clean.csv', f'../export_data/food/food_scenario_{i}.csv', error_percentages)

food_missing_ecar_0.csv deviates from intended error_pct by absolute 0.0%.
food_missing_ecar_1.csv deviates from intended error_pct by absolute 0.0%.
food_missing_ecar_2.csv deviates from intended error_pct by absolute 0.0%.
food_missing_ecar_3.csv deviates from intended error_pct by absolute 0.0%.
food_missing_ecar_4.csv deviates from intended error_pct by absolute 0.0%.
food_missing_ecar_5.csv deviates from intended error_pct by absolute 0.0%.
food_missing_ecar_6.csv deviates from intended error_pct by absolute 0.0%.
food_missing_ecar_7.csv deviates from intended error_pct by absolute 0.0%.
food_missing_ecar_8.csv deviates from intended error_pct by absolute 0.0%.
food_missing_ecar_9.csv deviates from intended error_pct by absolute 0.0%.
food_scenario_0.csv deviates from intended error_pct by absolute 0.0%.
food_scenario_1.csv deviates from intended error_pct by absolute 0.0%.
food_scenario_2.csv deviates from intended error_pct by absolute 0.0%.
food_scenario_3.csv deviates from int

## rayyan

In [32]:
df_clean = read_csv_dataset('../data/rayyan/clean.csv')
df_dirty = read_csv_dataset('../data/rayyan/dirty.csv')

df_clean.to_csv(f'../export_data/rayyan/clean.csv', index=False)
df_dirty.to_csv(f'../export_data/rayyan/rayyan_original.csv', index=False)

error_percentages = (df_dirty != df_clean).sum() / df_dirty.shape[0]
create_mcar('rayyan', df_clean, error_percentages)

df_clean['article_title'] = df_clean['article_title'].astype('string')
df_clean['journal_title'] = df_clean['journal_title'].astype('string')
df_clean['journal_issn'] = df_clean['journal_issn'].astype('string')
df_clean['article_jvolumn'] = df_clean['article_jvolumn'].astype('category')
df_clean['article_jissue'] = df_clean['article_jissue'].astype('category')
df_clean['article_jcreated_at'] = df_clean['article_jcreated_at'].astype('category')
df_clean['article_pagination'] = df_clean['article_pagination'].astype('string')
df_clean['author_list'] = df_clean['author_list'].astype('category')



Saved MCAR dataset rayyan iteration 0
Saved MCAR dataset rayyan iteration 1
Saved MCAR dataset rayyan iteration 2
Saved MCAR dataset rayyan iteration 3
Saved MCAR dataset rayyan iteration 4
Saved MCAR dataset rayyan iteration 5
Saved MCAR dataset rayyan iteration 6
Saved MCAR dataset rayyan iteration 7
Saved MCAR dataset rayyan iteration 8
Saved MCAR dataset rayyan iteration 9


In [33]:
scenario_name = 'rayyan_scenario'

for i in range(10):
    base_seed = i * 1000  # Using 1000 ensures no overlap between iterations
    seeds = [base_seed + (j+1) for j in range(len(df_clean.columns))]
    
    mid_lvl_config = {
        "article_title": [ErrorModel(EAR(condition_to_column='journal_title', seed=seeds[0]), error_type.MissingValue({'missing_value': '?'}), error_percentages['article_title'])],
        "journal_title": [ErrorModel(ENAR(seed=seeds[1]), error_type.MissingValue({'missing_value': '?'}), error_percentages['journal_title'])],
        "journal_issn": [ErrorModel(EAR(condition_to_column='journal_issn', seed=seeds[2]), error_type.MissingValue({'missing_value': '?'}), error_percentages['journal_issn'])],
        "article_jvolumn": [ErrorModel(EAR(condition_to_column='journal_title', seed=seeds[3]), error_type.CategorySwap(), error_percentages['article_jvolumn'])],
        "article_jissue": [ErrorModel(EAR(condition_to_column='journal_title', seed=seeds[4]), error_type.CategorySwap(), error_percentages['article_jissue'])],
        "article_jcreated_at": [ErrorModel(EAR(condition_to_column='journal_title', seed=seeds[5]), error_type.CategorySwap(), error_percentages['article_jcreated_at'])],
        "article_pagination": [ErrorModel(EAR(condition_to_column='article_pagination', seed=seeds[6]), error_type.MissingValue({'missing_value': '?'}), error_percentages['article_pagination'])],
        "author_list": [ErrorModel(EAR(condition_to_column='article_pagination', seed=seeds[7]), error_type.CategorySwap(), error_percentages['author_list'])],
    }

    scenario = MidLevelConfig(mid_lvl_config)
    df_corrupted, error_mask = mid_level.create_errors(df_clean, scenario)
    df_corrupted.to_csv(f'../export_data/rayyan/{scenario_name}_{i}.csv', index=False)

In [34]:
for i in range(10):
    verify_error_pct('../export_data/rayyan/clean.csv', f'../export_data/rayyan/rayyan_missing_ecar_{i}.csv', error_percentages)
for i in range(10):
    verify_error_pct('../export_data/rayyan/clean.csv', f'../export_data/rayyan/rayyan_scenario_{i}.csv', error_percentages)

rayyan_missing_ecar_0.csv deviates from intended error_pct by absolute 0.0%.
rayyan_missing_ecar_1.csv deviates from intended error_pct by absolute 0.0%.
rayyan_missing_ecar_2.csv deviates from intended error_pct by absolute 0.0%.
rayyan_missing_ecar_3.csv deviates from intended error_pct by absolute 0.0%.
rayyan_missing_ecar_4.csv deviates from intended error_pct by absolute 0.0%.
rayyan_missing_ecar_5.csv deviates from intended error_pct by absolute 0.0%.
rayyan_missing_ecar_6.csv deviates from intended error_pct by absolute 0.0%.
rayyan_missing_ecar_7.csv deviates from intended error_pct by absolute 0.0%.
rayyan_missing_ecar_8.csv deviates from intended error_pct by absolute 0.0%.
rayyan_missing_ecar_9.csv deviates from intended error_pct by absolute 0.0%.
rayyan_scenario_0.csv deviates from intended error_pct by absolute 0.0%.
rayyan_scenario_1.csv deviates from intended error_pct by absolute 0.0%.
rayyan_scenario_2.csv deviates from intended error_pct by absolute 0.0%.
rayyan_scen

## restaurant

In [35]:
df_clean = read_csv_dataset('../data/restaurant/clean.csv')
df_dirty = read_csv_dataset('../data/restaurant/restaurant_3_1.csv')

df_clean.to_csv(f'../export_data/restaurant/clean.csv', index=False)
df_dirty.to_csv(f'../export_data/restaurant/restaurant_original.csv', index=False)

error_percentages = (df_dirty != df_clean).sum() / df_dirty.shape[0]
create_mcar('restaurant', df_clean, error_percentages)

df_clean['city'] = df_clean['city'].astype('string')
df_clean['phone'] = df_clean['phone'].astype('string')
df_clean['type'] = df_clean['type'].astype('category')
df_clean['class'] = df_clean['class'].astype('category')


Saved MCAR dataset restaurant iteration 0
Saved MCAR dataset restaurant iteration 1
Saved MCAR dataset restaurant iteration 2
Saved MCAR dataset restaurant iteration 3
Saved MCAR dataset restaurant iteration 4
Saved MCAR dataset restaurant iteration 5
Saved MCAR dataset restaurant iteration 6
Saved MCAR dataset restaurant iteration 7
Saved MCAR dataset restaurant iteration 8
Saved MCAR dataset restaurant iteration 9


In [36]:
scenario_name = 'restaurant_scenario'

for i in range(10):
    base_seed = i * 1000  # Using 1000 ensures no overlap between iterations
    seeds = [base_seed + (j+1) for j in range(len(df_clean.columns))]
    
    mid_lvl_config = {
        "city": [ErrorModel(EAR(condition_to_column='class', seed=seeds[0]), error_type.MissingValue({'missing_value': '?'}), error_percentages['city'])],
        "phone": [ErrorModel(EAR(condition_to_column='class', seed=seeds[1]), error_type.Permutate({'permutation_separator': '-'}), error_percentages['phone'])],
        "type": [ErrorModel(EAR(condition_to_column='class', seed=seeds[2]), error_type.CategorySwap(), error_percentages['type'])],
        "class": [ErrorModel(ENAR(seed=seeds[3]), error_type.CategorySwap(), error_percentages['class'])],
    }

    scenario = MidLevelConfig(mid_lvl_config)
    df_corrupted, error_mask = mid_level.create_errors(df_clean, scenario)
    df_corrupted.to_csv(f'../export_data/restaurant/{scenario_name}_{i}.csv', index=False)

In [37]:
for i in range(10):
    verify_error_pct('../export_data/restaurant/clean.csv', f'../export_data/restaurant/restaurant_missing_ecar_{i}.csv', error_percentages)
for i in range(10):
    verify_error_pct('../export_data/restaurant/clean.csv', f'../export_data/restaurant/restaurant_scenario_{i}.csv', error_percentages)

restaurant_missing_ecar_0.csv deviates from intended error_pct by absolute 0.0%.
restaurant_missing_ecar_1.csv deviates from intended error_pct by absolute 0.0%.
restaurant_missing_ecar_2.csv deviates from intended error_pct by absolute 0.0%.
restaurant_missing_ecar_3.csv deviates from intended error_pct by absolute 0.0%.
restaurant_missing_ecar_4.csv deviates from intended error_pct by absolute 0.0%.
restaurant_missing_ecar_5.csv deviates from intended error_pct by absolute 0.0%.
restaurant_missing_ecar_6.csv deviates from intended error_pct by absolute 0.0%.
restaurant_missing_ecar_7.csv deviates from intended error_pct by absolute 0.0%.
restaurant_missing_ecar_8.csv deviates from intended error_pct by absolute 0.0%.
restaurant_missing_ecar_9.csv deviates from intended error_pct by absolute 0.0%.
restaurant_scenario_0.csv deviates from intended error_pct by absolute 0.0%.
restaurant_scenario_1.csv deviates from intended error_pct by absolute 0.0%.
restaurant_scenario_2.csv deviates f

## tax

In [38]:
df_clean = read_csv_dataset('../data/tax/clean.csv').iloc[:1000, :]
df_dirty = read_csv_dataset('../data/tax/dirty.csv').iloc[:1000, :]

df_clean.to_csv(f'../export_data/tax/clean.csv', index=False)
df_dirty.to_csv(f'../export_data/tax/tax_original.csv', index=False)

error_percentages = (df_dirty != df_clean).sum() / df_dirty.shape[0]
create_mcar('tax', df_clean, error_percentages)

df_clean['state'] = df_clean['state'].astype('category')
df_clean['zip'] = df_clean['zip'].astype('category')
df_clean['marital_status'] = df_clean['marital_status'].astype('category')
df_clean['has_child'] = df_clean['has_child'].astype('category')
df_clean['rate'] = df_clean['rate'].astype('string')


Saved MCAR dataset tax iteration 0
Saved MCAR dataset tax iteration 1
Saved MCAR dataset tax iteration 2
Saved MCAR dataset tax iteration 3
Saved MCAR dataset tax iteration 4
Saved MCAR dataset tax iteration 5
Saved MCAR dataset tax iteration 6
Saved MCAR dataset tax iteration 7
Saved MCAR dataset tax iteration 8
Saved MCAR dataset tax iteration 9


In [39]:
scenario_name = 'tax_scenario'

for i in range(10):
    base_seed = i * 1000  # Using 1000 ensures no overlap between iterations
    seeds = [base_seed + (j+1) for j in range(len(df_clean.columns))]
    
    mid_lvl_config = {
        "f_name": [ErrorModel(EAR(condition_to_column='city', seed=seeds[0]), error_type.MissingValue({'missing_value': '?'}), error_percentages['f_name'])],
        "l_name": [ErrorModel(EAR(condition_to_column='city', seed=seeds[1]), error_type.MissingValue({'missing_value': '?'}), error_percentages['l_name'])],
        "state": [ErrorModel(EAR(condition_to_column='city', seed=seeds[2]), error_type.CategorySwap(), error_percentages['state'])],
        "zip": [ErrorModel(ENAR(seed=seeds[3]), error_type.CategorySwap(), error_percentages['zip'])],
        "marital_status": [ErrorModel(ENAR(seed=seeds[4]), error_type.CategorySwap(), error_percentages['marital_status'])],
        "has_child": [ErrorModel(ENAR(seed=seeds[5]), error_type.CategorySwap(), error_percentages['has_child'])],
        "rate": [ErrorModel(ENAR(seed=seeds[6]), error_type.Typo(), error_percentages['rate'])],
    }

    scenario = MidLevelConfig(mid_lvl_config)
    df_corrupted, error_mask = mid_level.create_errors(df_clean, scenario)
    df_corrupted.to_csv(f'../export_data/tax/{scenario_name}_{i}.csv', index=False)

In [40]:
for i in range(10):
    verify_error_pct('../export_data/tax/clean.csv', f'../export_data/tax/tax_missing_ecar_{i}.csv', error_percentages)
for i in range(10):
    verify_error_pct('../export_data/tax/clean.csv', f'../export_data/tax/tax_scenario_{i}.csv', error_percentages)

tax_missing_ecar_0.csv deviates from intended error_pct by absolute 0.0%.
tax_missing_ecar_1.csv deviates from intended error_pct by absolute 0.0%.
tax_missing_ecar_2.csv deviates from intended error_pct by absolute 0.0%.
tax_missing_ecar_3.csv deviates from intended error_pct by absolute 0.0%.
tax_missing_ecar_4.csv deviates from intended error_pct by absolute 0.0%.
tax_missing_ecar_5.csv deviates from intended error_pct by absolute 0.0%.
tax_missing_ecar_6.csv deviates from intended error_pct by absolute 0.0%.
tax_missing_ecar_7.csv deviates from intended error_pct by absolute 0.0%.
tax_missing_ecar_8.csv deviates from intended error_pct by absolute 0.0%.
tax_missing_ecar_9.csv deviates from intended error_pct by absolute 0.0%.
tax_scenario_0.csv deviates from intended error_pct by absolute 0.0%.
tax_scenario_1.csv deviates from intended error_pct by absolute 0.0%.
tax_scenario_2.csv deviates from intended error_pct by absolute 0.0%.
tax_scenario_3.csv deviates from intended error_pc

## Hospital

In [43]:
# load data
df_clean = read_csv_dataset('../data/hospital/clean.csv')
df_dirty = read_csv_dataset('../data/hospital/dirty.csv')


df_clean = df_clean.rename(columns={'StateAverage': 'Stateavg'})
df_dirty = df_dirty.rename(columns={'StateAverage': 'Stateavg'})

df_clean = df_clean.drop('index', axis=1)
df_dirty = df_dirty.drop('index', axis=1)

df_dirty.to_csv(f'../export_data/hospital/hospital_original.csv', index=False)

# export mcar scenario
error_percentages = (df_dirty != df_clean).sum() / df_dirty.shape[0]
create_mcar('hospital', df_clean, error_percentages)

# apply realistic dtypes
df_clean['ProviderNumber'] = df_clean['ProviderNumber'].astype('category')
df_clean['HospitalName'] = df_clean['HospitalName'].astype('str')
df_clean['Address1'] = df_clean['Address1'].astype('str')
df_clean['Address2'] = df_clean['Address2'].astype('str')
df_clean['Address3'] = df_clean['Address3'].astype('str')
df_clean['City'] = df_clean['City'].astype('str')
df_clean['State'] = df_clean['State'].astype('category')
df_clean['ZipCode'] = df_clean['ZipCode'].astype('category')
df_clean['CountyName'] = df_clean['CountyName'].astype('str')
df_clean['PhoneNumber'] = df_clean['PhoneNumber'].astype('str')
df_clean['HospitalType'] = df_clean['HospitalType'].astype('str')
df_clean['HospitalOwner'] = df_clean['HospitalOwner'].astype('category')
df_clean['EmergencyService'] = df_clean['EmergencyService'].astype('category')
df_clean['Condition'] = df_clean['Condition'].astype('category')
df_clean['MeasureCode'] = df_clean['MeasureCode'].astype('category')
df_clean['MeasureName'] = df_clean['MeasureName'].astype('str')
df_clean['Score'] = df_clean['Score'].astype('str')
df_clean['Sample'] = df_clean['Sample'].astype('str')
df_clean['Stateavg'] = df_clean['Stateavg'].astype('str')

df_clean.to_csv(f'../export_data/hospital/clean.csv', index=False)

Saved MCAR dataset hospital iteration 0
Saved MCAR dataset hospital iteration 1
Saved MCAR dataset hospital iteration 2
Saved MCAR dataset hospital iteration 3
Saved MCAR dataset hospital iteration 4
Saved MCAR dataset hospital iteration 5
Saved MCAR dataset hospital iteration 6
Saved MCAR dataset hospital iteration 7
Saved MCAR dataset hospital iteration 8
Saved MCAR dataset hospital iteration 9


In [44]:
scenario_name = 'hospital_scenario'

for i in range(10):
    base_seed = i * 1000  # Using 1000 ensures no overlap between iterations
    seeds = [base_seed + (j+1) for j in range(len(df_clean.columns))]
    
    mid_lvl_config = {
        "ProviderNumber": [ErrorModel(EAR(condition_to_column='State', seed=seeds[0]), error_type.MissingValue(seed=seeds[0]), error_percentages['ProviderNumber'])],
        "HospitalName": [ErrorModel(EAR(condition_to_column='State', seed=seeds[1]), error_type.MissingValue(seed=seeds[1]), error_percentages['HospitalName'])],
        "Address1": [ErrorModel(EAR(condition_to_column='State', seed=seeds[2]), error_type.Typo(seed=seeds[2]), error_percentages['Address1'])],
        "City": [ErrorModel(EAR(condition_to_column='State', seed=seeds[3]), error_type.Typo(seed=seeds[3]), error_percentages['City'])],
        "State": [ErrorModel(ENAR(seed=seeds[4]), error_type.CategorySwap(seed=seeds[4]), error_percentages['State'])],
        "ZipCode": [ErrorModel(EAR(condition_to_column='State', seed=seeds[5]), error_type.CategorySwap(seed=seeds[5]), error_percentages['ZipCode'])],
        "CountyName": [ErrorModel(EAR(condition_to_column='State', seed=seeds[6]), error_type.Typo(seed=seeds[6]), error_percentages['CountyName'])],
        "PhoneNumber": [ErrorModel(EAR(condition_to_column='State', seed=seeds[7]), error_type.MissingValue(seed=seeds[7]), error_percentages['PhoneNumber'])],
        "HospitalType": [ErrorModel(EAR(condition_to_column='State', seed=seeds[8]), error_type.Typo(seed=seeds[8]), error_percentages['HospitalType'])],
        "HospitalOwner": [ErrorModel(EAR(condition_to_column='State', seed=seeds[9]), error_type.CategorySwap(seed=seeds[9]), error_percentages['HospitalOwner'])],
        "EmergencyService": [ErrorModel(EAR(condition_to_column='State', seed=seeds[10]), error_type.CategorySwap(seed=seeds[10]), error_percentages['EmergencyService'])],
        "Condition": [ErrorModel(EAR(condition_to_column='State', seed=seeds[11]), error_type.CategorySwap(seed=seeds[11]), error_percentages['Condition'])],
        "MeasureCode": [ErrorModel(EAR(condition_to_column='State', seed=seeds[12]), error_type.CategorySwap(seed=seeds[12]), error_percentages['MeasureCode'])],
        "MeasureName": [ErrorModel(EAR(condition_to_column='State', seed=seeds[13]), error_type.Typo(seed=seeds[13]), error_percentages['MeasureName'])],
        "Score": [ErrorModel(EAR(condition_to_column='State', seed=seeds[14]), error_type.Typo(seed=seeds[14]), error_percentages['Score'])],
        "Sample": [ErrorModel(EAR(condition_to_column='State', seed=seeds[15]), error_type.Typo(seed=seeds[15]), error_percentages['Sample'])],
        "Stateavg": [ErrorModel(EAR(condition_to_column='State', seed=seeds[16]), error_type.MissingValue(seed=seeds[16]), error_percentages['Stateavg'])],
    }

    scenario = MidLevelConfig(mid_lvl_config)
    df_corrupted, error_mask = mid_level.create_errors(df_clean, scenario)
    df_corrupted.to_csv(f'../export_data/hospital/{scenario_name}_{i}.csv', index=False)

In [45]:
for i in range(10):
    verify_error_pct('../export_data/hospital/clean.csv', f'../export_data/hospital/hospital_missing_ecar_{i}.csv', error_percentages)
for i in range(10):
    verify_error_pct('../export_data/hospital/clean.csv', f'../export_data/hospital/hospital_scenario_{i}.csv', error_percentages)

hospital_missing_ecar_0.csv deviates from intended error_pct by absolute 0.0%.
hospital_missing_ecar_1.csv deviates from intended error_pct by absolute 0.0%.
hospital_missing_ecar_2.csv deviates from intended error_pct by absolute 0.0%.
hospital_missing_ecar_3.csv deviates from intended error_pct by absolute 0.0%.
hospital_missing_ecar_4.csv deviates from intended error_pct by absolute 0.0%.
hospital_missing_ecar_5.csv deviates from intended error_pct by absolute 0.0%.
hospital_missing_ecar_6.csv deviates from intended error_pct by absolute 0.0%.
hospital_missing_ecar_7.csv deviates from intended error_pct by absolute 0.0%.
hospital_missing_ecar_8.csv deviates from intended error_pct by absolute 0.0%.
hospital_missing_ecar_9.csv deviates from intended error_pct by absolute 0.0%.
hospital_scenario_0.csv deviates from intended error_pct by absolute 0.19999999999999984%.
hospital_scenario_1.csv deviates from intended error_pct by absolute 0.8999999999999999%.
hospital_scenario_2.csv devia